In [5]:
# 라이브러리 import
from selenium import webdriver
import time
import pandas as pd
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager

In [6]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--disable-dex-shm-usage')

keys = Keys()

In [7]:
url_list = ['https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B7%80%EB%86%8D']

driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


C:\Users\user\AppData\Local\Temp\ipykernel_26608\2250834565.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
C:\Users\user\AppData\Local\Temp\ipykernel_26608\2250834565.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


In [8]:
all_count=0
text_list= []
star_list= []
date_list= []

for target_url in url_list:
    end = False
    
    driver.get(target_url)
    driver.implicitly_wait(3)
    time.sleep(1)
    count = 0
    
    review_num = driver.find_element('xpath', '//*[@id="content"]/div/div[2]/div[1]/div[2]/div[1]/a/strong')
    review_num = review_num.text
    review_num = re.sub('[^#0-9 ]',"", review_num)
    review_num = int(review_num)
    
    try:
        driver.find_element('xpath', '//*[@id="content"]/div/div[3]/div[3]/ul/li[2]/a').click()
    except:
        driver.find_element('xpath', '//*[@id="content"]/div/div[3]/div[4]/ul/li[2]').click()
    
    time.sleep(1)
    
    while(True):
        for i in range(20):
            # 공통 경로
            path = driver.find_element('xpath', f'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li[{i+1}]/div')
            try: # 판매자 댓글 없을때                    
                review = path.find_element('xpath', 'div/div/div[1]/div/div[1]/div[2]/div')
                star = path.find_element('xpath', 'div/div/div[1]/div/div[1]/div[1]/div[2]/div[1]/em')
                date = path.find_element('xpath', 'div/div/div[1]/div/div[1]/div[1]/div[2]/div[2]/span')

            except:
                try: # 판매자 댓글 있을때
                    review = path.find_element('xpath', 'div[1]/div/div[1]/div/div[1]/div[2]/div')
                    star = path.find_element('xpath', 'div[1]/div/div[1]/div/div[1]/div[1]/div[2]/div[1]/em')
                    date = path.find_element('xpath', 'div[1]/div/div[1]/div/div/div[1]/div[2]/div[2]/span')

                except: # 경로가 다른경우
                    review = path.find_element('xpath', 'div/div/div[1]/div/div/div[2]/div')
                    star = path.find_element('xpath', 'div/div/div[1]/div/div/div[1]/div[2]/div[1]/em')
                    date = path.find_element('xpath', 'div/div/div[1]/div/div/div[1]/div[2]/div[2]/span')
                    
            r_span_num = len(review.find_elements('xpath','span'))
            
            if r_span_num == 1: # BEST, 한달사용기 등이 없을때
                review = review.find_element('xpath', 'span')
            else: # 있을때
                review = review.find_element('xpath', f'span[{r_span_num}]')
                
            review = review.text
            review = re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣 ]',"", review)
            star = star.text
            star = re.sub('[^#0-9 ]',"", star)
            date = date.text

            text_list.append(review)
            star_list.append(star)
            date_list.append(date)
            
            count = count + 1
            all_count = all_count + 1
            
            if count == review_num:
                end = True
                print('{}개 리뷰 크롤링, 다음'.format(count))
                break
        
        print('총 {}개 리뷰 크롤링'.format(all_count))
        
        if end:
            break
        
        next = 12
        next_bool = True
        while(next_bool): # 다음 버튼 클릭
            try:
                driver.find_element('xpath', f'//*[@id="REVIEW"]/div/div[3]/div[2]/div/div/a[{next}]').click()
                next_bool = False
            except:
                next = next -1
                
        driver.implicitly_wait(3)
        time.sleep(1)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/div/div[2]/div[1]/div[2]/div[1]/a/strong"}
  (Session info: chrome=108.0.5359.126)
Stacktrace:
Backtrace:
	(No symbol) [0x0096F243]
	(No symbol) [0x008F7FD1]
	(No symbol) [0x007ED04D]
	(No symbol) [0x0081C0B0]
	(No symbol) [0x0081C22B]
	(No symbol) [0x0084E612]
	(No symbol) [0x008385D4]
	(No symbol) [0x0084C9EB]
	(No symbol) [0x00838386]
	(No symbol) [0x0081163C]
	(No symbol) [0x0081269D]
	GetHandleVerifier [0x00C09A22+2655074]
	GetHandleVerifier [0x00BFCA24+2601828]
	GetHandleVerifier [0x00A18C0A+619850]
	GetHandleVerifier [0x00A17830+614768]
	(No symbol) [0x009005FC]
	(No symbol) [0x00905968]
	(No symbol) [0x00905A55]
	(No symbol) [0x0091051B]
	BaseThreadInitThunk [0x75EE7D69+25]
	RtlInitializeExceptionChain [0x770BBB9B+107]
	RtlClearBits [0x770BBB1F+191]


In [53]:
len(text_list), len(star_list)

(13022, 13022)

In [54]:
data = {'date' : date_list,
        'star' : star_list,
        'text' : text_list
        }

df = pd.DataFrame(data)

df['date'] = pd.to_datetime(('20' + df['date']))

len(df)

13022

In [55]:
df

,date,star,text
0,2022-11-22,5,표문 막걸리는 제가 제일 좋아하는 막걸리예요표문막걸리 런칭 첫 라이브 에서부터 편의...
1,2022-11-22,5,와라이브 이벤트에 후디 당첨되었어요와 진짜 당첨될줄 몰랐는데 너무 기분이 좋네요후디...
2,2022-11-26,5,한강주조를 처음 알게된건 인사동 어느 고기집이었는데 부모님과 함께 간단히 마시려고 ...
3,2022-11-28,5,축구경기 보며 마셨어요부드럽고 은은한 단맛이 너무 좋아요계속 생각나는 맛이네요 라이...
4,2022-11-24,5,가격이 많이 내렸네여곰표 막걸리 중독성있습니다쏘는맛없지만 부드럽고 맛있어요
...,...,...,...
13017,2021-04-27,5,
13018,2021-04-25,5,캠핑와서 먹었어요부드럽게 잘 넘어가요
13019,2021-04-07,5,감사합니다 또감사합니다
13020,2022-04-30,5,다른 막걸리랑 달라요도특하고 맛있어요


In [56]:
df.dropna(inplace=True) # 글이없는 리뷰 드랍
df.reset_index(drop=True, inplace=True)
df

,date,star,text
0,2022-11-22,5,표문 막걸리는 제가 제일 좋아하는 막걸리예요표문막걸리 런칭 첫 라이브 에서부터 편의...
1,2022-11-22,5,와라이브 이벤트에 후디 당첨되었어요와 진짜 당첨될줄 몰랐는데 너무 기분이 좋네요후디...
2,2022-11-26,5,한강주조를 처음 알게된건 인사동 어느 고기집이었는데 부모님과 함께 간단히 마시려고 ...
3,2022-11-28,5,축구경기 보며 마셨어요부드럽고 은은한 단맛이 너무 좋아요계속 생각나는 맛이네요 라이...
4,2022-11-24,5,가격이 많이 내렸네여곰표 막걸리 중독성있습니다쏘는맛없지만 부드럽고 맛있어요
...,...,...,...
13017,2021-04-27,5,
13018,2021-04-25,5,캠핑와서 먹었어요부드럽게 잘 넘어가요
13019,2021-04-07,5,감사합니다 또감사합니다
13020,2022-04-30,5,다른 막걸리랑 달라요도특하고 맛있어요


In [57]:
df[df.text.str.contains('BEST|한달사용기|재구매')] # 머릿말 들어갔는지 확인

,date,star,text
15,2022-11-22,5,월드컵 응원하기 위해 재구매합니다ㅎㅎ
25,2022-10-27,5,맛있었어요 재구매 할게요
64,2022-10-24,5,맛있게 잘 먹었어요재구매 할겁니다
76,2022-09-15,5,재구매해서 먹고 맛있어서 또 시켰습니다 적당히 달달하고 목넘김 좋아요
120,2022-08-03,1,그냥 가격만 비싸고 대박이나 장수가 훨씬 맛있네요 가격도 500m에 4천원꼴인데 재...
...,...,...,...
12852,2021-04-29,5,재구매 또 했어용 9시만 기다려용
12894,2021-04-13,1,배송 최악가격대비 양 챵렬맛 평범재구매 의사 없음
12898,2021-04-21,5,어제 받았습니다넘 기대되네요맛있으면 재구매하려구요
12935,2021-04-13,5,역시 곰표 재구매 할거에요넘 맛있어요ㅜㅜ


In [58]:
df[df.text.duplicated(keep= False)] # 중복 확인

# df.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거

,date,star,text
20,2022-11-19,5,배송도 빠르고 좋네요
36,2022-11-07,4,부드럽고 맛있네요
43,2022-11-02,5,빠르게 잘받았습니다
44,2022-10-30,5,빠른 배송 감사합니다
45,2022-10-30,5,빠른 배송 감사합니다
...,...,...,...
13003,2021-05-01,5,너무너무너무 좋아요
13013,2021-04-07,5,달달하니 맛있어요
13016,2021-04-28,5,
13017,2021-04-27,5,


In [59]:
df

,date,star,text
0,2022-11-22,5,표문 막걸리는 제가 제일 좋아하는 막걸리예요표문막걸리 런칭 첫 라이브 에서부터 편의...
1,2022-11-22,5,와라이브 이벤트에 후디 당첨되었어요와 진짜 당첨될줄 몰랐는데 너무 기분이 좋네요후디...
2,2022-11-26,5,한강주조를 처음 알게된건 인사동 어느 고기집이었는데 부모님과 함께 간단히 마시려고 ...
3,2022-11-28,5,축구경기 보며 마셨어요부드럽고 은은한 단맛이 너무 좋아요계속 생각나는 맛이네요 라이...
4,2022-11-24,5,가격이 많이 내렸네여곰표 막걸리 중독성있습니다쏘는맛없지만 부드럽고 맛있어요
...,...,...,...
13017,2021-04-27,5,
13018,2021-04-25,5,캠핑와서 먹었어요부드럽게 잘 넘어가요
13019,2021-04-07,5,감사합니다 또감사합니다
13020,2022-04-30,5,다른 막걸리랑 달라요도특하고 맛있어요


In [60]:
df.to_csv("술 리뷰(곰표막걸리).csv", encoding="utf-8-sig", index=False)